In [1]:
import gym
import ray

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [3]:
# Initialize Ray
ray.shutdown()
ray.init(ignore_reinit_error=True)

2020-09-06 14:26:36,944	INFO resource_spec.py:231 -- Starting Ray with 14.16 GiB memory available for workers and up to 7.09 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-06 14:26:37,482	INFO services.py:1193 -- View the Ray dashboard at localhost:8267


{'node_ip_address': '192.168.7.73',
 'raylet_ip_address': '192.168.7.73',
 'redis_address': '192.168.7.73:24871',
 'object_store_address': '/tmp/ray/session_2020-09-06_14-26-36_943216_21585/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-06_14-26-36_943216_21585/sockets/raylet',
 'webui_url': 'localhost:8267',
 'session_dir': '/tmp/ray/session_2020-09-06_14-26-36_943216_21585'}

In [4]:
print("Dashboard URL: http://{}".format(ray.get_webui_url()))

Dashboard URL: http://localhost:8267


### Register MineRL Gym Environment to RLlib

In [5]:
import minerl
from gym import envs

/home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
from minerl_rllib.envs.env import MineRLRandomDebugEnv

In [7]:
# Register MineRL Gym Environment to RLLIB
# https://docs.ray.io/en/latest/rllib-env.html
from ray.tune.registry import register_env

def minerl_env_creator(env_config):
    import minerl
    
    if 'minerl_env_name' in env_config:
        # TODO use logger
        print('MineRL Env Name found...')
        env_name = env_config['minerl_env_name']
    else:
        # TODO use logger
        print('No MineRL Env name specified, using MineRLNavigateDense-v0')
        env_name = 'MineRLNavigateDense-v0'
        
        
#     # Check minerl environments are imported
#     # This only checks Gym environment not the ones registered in Ray!
#     all_envs = envs.registry.all()
#     env_ids = [env_spec.id for env_spec in all_envs]
#     print(env_ids)

# This doesnt work, need wrapper to discretize the action space for DQN
#     minerl_env = gym.make(env_name) 
    
    # Second version but has .nan rewards
#     core_env = gym.make(env_name) # A MineRLNavigate-v0 env
#     minerl_env = wrap_env(core_env, test=False)
    
#     minerl_env = gym.make('MineRLNavigateDense-v0')
    
    return minerl_env  

register_env("minerl", minerl_env_creator)

In [11]:
from minerl_rllib.envs import register

In [12]:
register()

Registering Environment Name: MineRLTreechopVectorObf-v0
Registering Environment Name: MineRLNavigateVectorObf-v0
Registering Environment Name: MineRLNavigateExtremeVectorObf-v0
Registering Environment Name: MineRLNavigateDenseVectorObf-v0
Registering Environment Name: MineRLNavigateExtremeDenseVectorObf-v0
Registering Environment Name: MineRLObtainDiamondVectorObf-v0
Registering Environment Name: MineRLObtainDiamondDenseVectorObf-v0
Registering Environment Name: MineRLObtainIronPickaxeVectorObf-v0
Registering Environment Name: MineRLObtainIronPickaxeDenseVectorObf-v0
Registering Environment Name: MineRLObtainDiamondSurvivalVectorObf-v0


In [13]:
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "use_pytorch": True,
                 'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 },
         stop={"training_iteration": 2, "timesteps_total": 1000})  
# Config notes:
# "log_level": "INFO" for verbose,
# "eager": True for eager execution,

Instructions for updating:
non-resource variables are not supported in the long term


Trial name,status,loc
PPO_MineRLNavigateDenseVectorObf-v0_b353d_00000,RUNNING,


(pid=21679) WARNING:tensorflow:From /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=21679) Instructions for updating:
(pid=21679) non-resource variables are not supported in the long term
(pid=21679) /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=21679)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=21679) 2020-09-06 14:27:03,542	WARNING deprecation.py:30 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
(pid=21679) 2020-09-06 14:27:03,542	INFO trainer.py:632 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=21679) 

Result for PPO_MineRLNavigateDenseVectorObf-v0_b353d_00000:
  custom_metrics: {}
  date: 2020-09-06_14-33-14
  done: true
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 89e50cbf5e214e44a24b27cea0b9633f
  experiment_tag: '0'
  hostname: blackbox
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 90.88575339317322
        entropy_coeff: 0.0
        kl: 0.032430309103801847
        policy_loss: -0.060248322144616395
        total_loss: 0.2571080564521253
        vf_explained_var: 0.017088599503040314
        vf_loss: 0.3108703177422285
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.7.73
  num_healthy_workers: 2
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 58.006553911205074
    ram_util_percent: 67.83551797040168

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_b353d_00000,TERMINATED,,1,331.35,4000,nan


Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_b353d_00000,TERMINATED,,1,331.35,4000,nan


(pid=21679) *** Aborted at 1599427997 (unix time) try "date -d @1599427997" if you are using GNU date ***
(pid=21679) PC: @                0x0 (unknown)
(pid=21679) *** SIGSEGV (@0x7fd308ff99d0) received by PID 22028 (TID 0x7fd32b8df740) from PID 150968784; stack trace: ***
(pid=21679)     @     0x7fd32bc503c0 (unknown)
(pid=21679)     @     0x7fd32bc45aab __pthread_clockjoin_ex
(pid=21679)     @     0x7fd32981b2d3 std::thread::join()
(pid=21679)     @     0x7fd329d4c493 ray::gcs::GlobalStateAccessor::Disconnect()
(pid=21679)     @     0x7fd329bebfbc __pyx_pw_3ray_7_raylet_19GlobalStateAccessor_5disconnect()
(pid=21679)     @     0x5604de6dbb71 _PyMethodDef_RawFastCallKeywords
(pid=21679)     @     0x5604de6e2aef _PyMethodDescr_FastCallKeywords
(pid=21679)     @     0x5604de74737c _PyEval_EvalFrameDefault
(pid=21679)     @     0x5604de6db20b _PyFunction_FastCallKeywords
(pid=21679)     @     0x5604de742e70 _PyEval_EvalFrameDefault
(pid=21679)     @     0x5604de68b2b9 _PyEval_EvalCodeWi